<a href="https://colab.research.google.com/github/Omar-Works/ID-2024/blob/main/vgg16_test14_run7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Connecting to **Malariav2** dataset

In [34]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Evaluates model performance

In [35]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, matthews_corrcoef

def find_result():
    predictions = model.predict(test_generator)
    predictions = (predictions > 0.5).astype(int).flatten()  # Convert to binary labels

    # True labels
    y_true = test_generator.classes

    # Calculate metrics
    accuracy = accuracy_score(y_true, predictions)
    classification_error = 1 - accuracy
    precision = precision_score(y_true, predictions)
    recall = recall_score(y_true, predictions)
    f1 = f1_score(y_true, predictions)
    specificity = recall_score(y_true, predictions, pos_label=0)
    fall_out = 1 - specificity
    mcc = matthews_corrcoef(y_true, predictions)

    # Print metrics
    print("Accuracy:", accuracy)
    print("Classification Error:", classification_error)
    print("Precision:", precision)
    print("Recall (Sensitivity):", recall)
    print("F1 Score:", f1)
    print("Specificity:", specificity)
    print("Fall-Out:", fall_out)
    print("Matthews Correlation Coefficient:", mcc)
    print("Confusion Matrix:\n", confusion_matrix(y_true, predictions))
    print("Classification Report:\n", classification_report(y_true, predictions))


In [36]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Unfreeze the base model
base_model.trainable = True

# Optionally freeze some of the earlier layers to retain general features
for layer in base_model.layers[:10]:  # You can adjust this number based on experimentation
    layer.trainable = False

# Create the model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # For binary classification
])


In [37]:
# Data Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=20, width_shift_range=0.2,
    height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode='nearest', validation_split=0.2)  # Added validation split

# Use separate subsets for training and validation
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Malariav2/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'  # Specify training subset
)

validation_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Malariav2/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'  # Specify validation subset
)

# Evaluate the model on the test dataset
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Malariav2/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False  # Ensure order for metric calculations
)

Found 333 images belonging to 2 classes.
Found 83 images belonging to 2 classes.
Found 134 images belonging to 2 classes.


In [38]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),  # Low learning rate
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define the early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=3, restore_best_weights=True
)

history = model.fit(train_generator, epochs=10, validation_data=validation_generator, callbacks=[early_stopping])


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


11/11 ━━━━━━━━━━━━━━━━━━━━ 16s 768ms/step - accuracy: 0.5334 - loss: 0.6888 - val_accuracy: 0.5904 - val_loss: 0.6566
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 15s 486ms/step - accuracy: 0.7262 - loss: 0.6372 - val_accuracy: 0.6747 - val_loss: 0.5899
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 404ms/step - accuracy: 0.8621 - loss: 0.5534 - val_accuracy: 0.8795 - val_loss: 0.4578
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 461ms/step - accuracy: 0.8354 - loss: 0.4601 - val_accuracy: 0.8554 - val_loss: 0.3608
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 8s 512ms/step - accuracy: 0.9098 - loss: 0.3063 - val_accuracy: 0.9398 - val_loss: 0.2073
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 10s 433ms/step - accuracy: 0.9491 - loss: 0.1843 - val_accuracy: 0.9518 - val_loss: 0.1538
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 511ms/step - accuracy: 0.9540 - loss: 0.1538 - val_accuracy: 0.9759 - val_loss: 0.0808
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 9s 579ms/step - accuracy: 0.9393 - loss: 0.1544 - val_accuracy: 0.9759 - 

In [39]:
find_result()

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step
Accuracy: 0.9402985074626866
Classification Error: 0.05970149253731338
Precision: 0.8571428571428571
Recall (Sensitivity): 0.9767441860465116
F1 Score: 0.9130434782608695
Specificity: 0.9230769230769231
Fall-Out: 0.07692307692307687
Matthews Correlation Coefficient: 0.8721749283807382
Confusion Matrix:
 [[84  7]
 [ 1 42]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.92      0.95        91
           1       0.86      0.98      0.91        43

    accuracy                           0.94       134
   macro avg       0.92      0.95      0.93       134
weighted avg       0.95      0.94      0.94       134



In [42]:
# don't run this again
# Save the entire model
model.save('/content/drive/MyDrive/saved_models/vgg16-final_model.keras')  # Save to your Google Drive

In [41]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report